In [1]:
# Softmax function
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    # print('after reshaping the data', X_train.shape)
    
    # Normalize the data: subtract the mean image - zero centered the data
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    #print('after Mean subtraction', X_train.shape)
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    # print('after adding bias dimension', X_train.shape)
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


In [28]:
import numpy as np
from random import shuffle
def softmax_loss_naive(W, X, y, reg):
    
    # initialize the loss and gradient to zero
    loss = 0.0
    dW = np.zeros_like(W)
    
    # compute the softmax loss and gradient
    num_train = X.shape[0]
    num_classes = W.shape[1]
    loss = 0.0
    for i in range(num_train):
        # compute vector for scores
        score = X[i].dot(W)
        
        # Normailization trick to avoid numerical unstability
        score -= np.max(score)
        
        # compute loss
        sum_j = np.sum(np.exp(score))
        p = lambda k : np.exp(score[k]) / sum_j
        loss += -np.log(p(y[i]))
        
        # compute gradient
        for k in range(num_classes):
            p_k = p(k)
            dW[:, k] += (p_k - (k == y[i])) * X[i]
            
        loss /= num_train
        loss += 0.5 * reg * np.sum(W * W)
        dW /= num_train
        dW += reg * W
    
    return loss, dW

In [33]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

#from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 0.004885
sanity check: 2.302585


In [34]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.120531 analytic: 0.120531, relative error: 2.644472e-09
numerical: -0.017558 analytic: -0.017558, relative error: 3.903135e-08
numerical: -0.007228 analytic: -0.007228, relative error: 7.841789e-09
numerical: -0.000918 analytic: -0.000918, relative error: 5.392355e-10
numerical: -0.017936 analytic: -0.017936, relative error: 6.447938e-08
numerical: -0.010466 analytic: -0.010466, relative error: 1.160397e-08
numerical: 0.009619 analytic: 0.009619, relative error: 8.591705e-09
numerical: -0.002945 analytic: -0.002945, relative error: 4.221054e-09
numerical: -0.013473 analytic: -0.013473, relative error: 1.905318e-08
numerical: -0.002836 analytic: -0.002836, relative error: 3.974989e-09
numerical: 0.036686 analytic: 0.036686, relative error: 4.570819e-10
numerical: 0.016566 analytic: 0.016566, relative error: 1.119113e-10
numerical: 0.003306 analytic: 0.003306, relative error: 3.226239e-09
numerical: -0.011667 analytic: -0.011667, relative error: 2.695931e-08
numerical: -0.03

In [40]:
import numpy as np
def softmax_loss_vectorized(W, X, y, reg):
    # initialize loss and gradient to zero
    loss = 0.0
    dW = np.zeros_like(W)
    
    # compute score vector
    num_train = X.shape[0]
    score = X.dot(W)
    # max of every sample
    score -= np.max(score, axis=1, keepdims=True)
    sum_score = np.sum(np.exp(score), axis=1, keepdims=True)
    p = np.exp(score) / sum_score
    loss = np.sum(-np.log(p[np.arange(num_train), y]))
              
    ind = np.zeros_like(p)
    ind[np.arange(num_train), y] = 1
    dW = X.T.dot(p - ind)

    loss /= num_train
    loss += 0.5 * reg * np.sum(W * W)
    dW /= num_train
    dW += reg*W
    
    return loss, dW


In [41]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

#from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 4.885380e-03 computed in 0.289091s
vectorized loss: 2.364039e+00 computed in 0.007278s
Loss difference: 2.359153
Gradient difference: 314.687813


In [53]:
class Softmax:
    def __init__(self, W):
        self.W = W

    def train(self, X, y, epochs, lr, batch_size, reg, verbose):
        num_examples = X.shape[0]
    # p = np.random.permutation(num_examples)
    # X = X[p]
    # y = y[p]
        for epoch in range(epochs):
            avg_loss = 0
            for i in range(0,num_examples, batch_size):
                batch_start = i
                batch_end = min(batch_start + batch_size, num_examples)
                x_batch = X[batch_start:batch_end]
                y_batch = y[batch_start:batch_end]
                loss, dW = softmax_loss_vectorized(self.W, x_batch, y_batch, reg)
                avg_loss+=loss
                self.W -= lr*dW
                avg_loss/= math.ceil(num_examples/batch_size)

        if epoch%verbose==0:
            print("Epoch {}, avg loss: {}".format(epoch, avg_loss))
  
    def classify(self, X):
        out = X@self.W
        return out

    def predict(self, X):
        out = X@self.W
        return np.argmax(out, axis=1)

In [57]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
#from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
iters = 100
for lr in learning_rates:
    for rs in regularization_strengths:
        softmax = Softmax()
        softmax.train(X_train, y_train, learning_rate=lr, reg=rs, num_iters=100)
        
        y_train_pred = softmax.predict(X_train)
        acc_train = np.mean(y_train == y_train_pred)
        
        y_val_pred = softmax.predict(X_val)
        acc_val = np.mean(y_val == y_val_pred)
        
        results[(lr, rs)] = (acc_train, acc_val)
        
        if best_val < acc_val:
            best_val = acc_val
            best_softmax = softmax

################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

AttributeError: 'NoneType' object has no attribute 'shape'